In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "arvc-cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, randint
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.00, 2.5),
        "--max_iter": randint(250)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", compute_target=cpu_cluster, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling=ps, policy=policy, primary_metric_name="Accuracy", 
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=50, max_concurrent_runs=10, max_duration_minutes=30)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdr = exp.submit(config=hyperdrive_config)

RunDetails(hdr).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hdr.get_best_run_by_primary_metric()

if best_run is None: 
    raise Exception("No best run was found")

parameter_values = best_run.get_details()['runDefinition']['arguments']

print(parameter_values)

os.makedirs("best_run_model_files", exist_ok=True)

for f in best_run.get_file_names():
    if f == "outputs/model.pkl":
        best_run.download_file(name=f, output_file_path = os.path.join("best_run_model_files", os.path.basename(f)))

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv",validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#combine x,y back into a single pandas dataframe
import pandas as pd
df = x.merge(y.rename('y'), left_index=True, right_index=True)
os.makedirs("data", exist_ok=True)
local_path = "data/prepared.csv"

#save pandas dataframe to csv
df.to_csv(local_path)

#upload csv to azure data store
datastore = ws.get_default_datastore()
datastore.upload(src_dir="data", target_path="data")

#read the new csv from Datastore as Tabulardataset
from azureml.core import Dataset
tabulardataset = Dataset.Tabular.from_delimited_files(path=[(datastore, ("data/prepared.csv"))])

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=tabulardataset,
    label_column_name='y',
    compute_target = cpu_cluster,
    n_cross_validations=10,
    max_concurrent_iterations=25)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=False)

In [ ]:
#this step is necessary because the version of azureml-train-automl-runtime on the jupyter environment does not match 
#the version used by AutoML
!pip uninstall -y azureml-train-automl-runtime
!pip install  azureml-train-automl-runtime==1.20.0

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()
os.makedirs("best_run_model_files", exist_ok=True)
joblib.dump(fitted_model, os.path.join("best_run_model_files", "model.pkl"))


#print the hyperparameters of the best model
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

In [ ]:
#Delete the compute cluster
cpu_cluster.delete()